# Jerre on Air
This website will help you get hyped up for the upcomming punkrock festival Jera on Air in the Netherlands.\
You can listen and rate to songs of the artists that are coming to the festival.

## Filling up the database
The following notebook is needed to fill up the database with artists and their songs.\
It will scrape the artists from the website of the festival.\
With the names of artists it will call the youtube API to gather the songs.

## Setup
### Required packages
These are the required python packages that needs to be installed.
```
pip install google-api-python-client
pip install isodate
pip install Django
```
### Google API key
As a setup you will need to register een application on google to have an API key to call the youtube API.\
[Setup an account and application.](https://console.cloud.google.com/apis/dashboard)
In the credentials section make an API key.\
Enable the 'YouTube Data API v3' in API & services section.

Enter the API key in the following section.

In [ ]:
# Youtube API key
DEVELOPER_KEY = "YOUR_API_KEY"

In [ ]:
import googleapiclient.discovery
import googleapiclient.errors
import requests
import re
import isodate

# Youtube API information
api_service_name = "youtube"
api_version = "v3"

# API client
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey = DEVELOPER_KEY)

## Setup the database
There is a django backend that stores the artists, songs and rating you give locally.\
In the following section you have the urls and ways to put things in the database.\
When using another backend or url, you can adjust it here.

To go further you need to setup the database and run the backend server.
```
python ./backend/manage.py migrate
python ./backend/manage.py runserver
```

In [ ]:
song_url = "http://localhost:8000/api/songs/"
artist_url = "http://localhost:8000/api/artists/"

def put_artist_in_database(name, pic):
    body = {
        "name": name,
        "picture": pic
    }
    requests.post(artist_url, data = body)

def put_song_in_database(id, name, views, artist_id):
    print(name + " (" + views + ") --> " + id)
    
    body = {
            "title": name,
            "link": id,
            "views": views,
            "artist": artist_id
        }
    requests.post(song_url, data = body)

def get_artists():
    return requests.get(artist_url).json()

## Gather the artist
In this section we will gather the artists from the festival website.\
Here we use BeautifulSoup to grab the artists in the html of the website.\
Then we put the artist with its picture in our local database.

Chnage this if you want to gather artists from another source.

In [ ]:
from bs4 import BeautifulSoup

jera_on_air = "https://www.jeraonair.nl/nl/line-up"
page = requests.get(jera_on_air)

soup = BeautifulSoup(page.content, "html.parser")
line_up = soup.find(id="line-up")
artists = line_up.find_all("div", class_="item")

for artist in artists:
    pic = "https://www.jeraonair.nl" + artist.find("img")["src"]
    name = artist.find("span", class_="bandname").text
    put_artist_in_database(name, pic)

## Gather the songs
Here we gather some songs of the artists.\
We will search youtube with the name of the artist and take the first 10 items.

The implementation is easy and can be improved.\
What goes wrong is some artists their name are ambiguous and can be interpreted as something else.\
So this can gather unrelated youtube videos.\
Therefore the application has a *Delete* button for these cases.

In [ ]:
def search_youtube(query, amount=10):
    request = youtube.search().list(
        part="snippet",
        q=query,
        maxResults = amount,
        # order = "viewCount",
        type = "video",
        
    )
    return request.execute()["items"]

def search_video_ids(artist, amount=10):
    videos = search_youtube(artist, amount)
    return list(map(lambda x: x["id"]["videoId"], videos))
    
def clean_title(title, artist):
    artist_pattern = re.compile(artist, re.IGNORECASE)
    value = artist_pattern.sub("", title)
    value = value.replace(artist, "")
    value = value.replace("-", "")
    value = re.sub("\(.*?\)","", value)
    value = value.strip()
    return value

def video_details(ids):
    request = youtube.videos().list(
        part="id, snippet, statistics, contentDetails",
        id=ids
    )
    return request.execute()["items"]

    
def video_not_too_long(item):
    duration_value = item["contentDetails"]["duration"]
    minutes = isodate.parse_duration(duration_value).total_seconds() / 60
    return minutes < 10

def put_songs_in_database_for_artist(artist_id, artist, amount=10):
    ids = search_video_ids(artist, amount)
    items = video_details(ids)
    songs = filter(video_not_too_long, items)
    for song in songs:
        title = song["snippet"]["title"]
        name = clean_title(title, artist)
        id = song["id"]
        views = song["statistics"]["viewCount"]
        put_song_in_database(id, name, views, artist_id)

def fill_database_with_songs(songs_per_artist=10):
    artists = get_artists()
    for artist in artists:
        print(artist["name"])
        put_songs_in_database_for_artist(artist["id"], artist["name"], songs_per_artist)

## Fill it up!
You can indicate here how many songs it will search per artist.\
The amount put into the database can deviate because if the video is longer than 10 minutes, it will not be seen as a song and be ignored.

**Attention!** This will take some time. Dependent on how many songs you choose.

In [ ]:
fill_database_with_songs(10)